In [ ]:
import cv2
import numpy as np
 
img1 = cv2.imread('data/map1.png')
img2 = cv2.imread('data/map2.png')

cv2.imshow('1',img1)
cv2.imshow('2',img2)
#img1 = cv2.resize(img1,(640,480))
#img2 = cv2.resize(img2,(640,480))
 
img_gray1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
img_gray2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
 
#创建特征检测器
sift = cv2.SIFT_create()

#计算特征点和描述子
kp1,des1 = sift.detectAndCompute(img_gray1,None)
kp2,des2 = sift.detectAndCompute(img_gray2,None)
 
#创建特征匹配器
index_params = dict(algorithm=1,trees=5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params,search_params)
 
#对描述子进行特征匹配
matches = flann.knnMatch(des1,des2,k=2) #用的knnmatch匹配
 
 
goods = []   #选择两个匹配对象中好一些的保存下来
for (m,n) in matches:
    if m.distance < n.distance*0.7:
        goods.append(m)
         
print('goods',len(goods))
 
#把找到的匹配特征点保存在goods中，注意单应性矩阵要求最少4个点
if len(goods) >= 4:
    #把goods中的第一幅图和第二幅图的特征点坐标拿出来（坐标要float32且是三维矩阵类型  reshape(-1,1,2)） goods是Dmatch对象
    src_points = np.float32([kp1[m.queryIdx].pt for m in goods]).reshape(-1,1,2)
    des_points = np.float32([kp2[m.trainIdx].pt for m in goods]).reshape(-1,1,2)
   # print('des_points:',des_points)
     
    #根据匹配上的关键点去计算单应性矩阵  第一个图对变成第二个图的视角计算出来的单应性矩阵
    H,mask = cv2.findHomography(src_points,des_points,cv2.RANSAC,5) #参数5表示：允许有5个关键点的误差
     
#     #利用H矩阵的逆求解视角和img1特征匹配的点的img2图，并且img1没有像素
#     warpimg = cv2.warpPerspective(img2,np.linalg.inv(H),(img1.shape[1]+img2.shape[1],img1.shape[0]+img2.shape[0]))
#     direct = warpimg.copy()
#     direct[0:img1.shape[0],0:img1.shape[1]] = img1 #将左边的img1的部分重新赋值
else:
    exit()
 
     
#获取原始图像的高宽
h1,w1 = img1.shape[:2]
h2,w2 = img2.shape[:2]
# 获取两幅图的边界坐标
img1_pts = np.float32([[0,0],[0,h1-1],[w1-1,h1-1],[w1-1,0]]).reshape(-1,1,2)
img2_pts = np.float32([[0,0],[0,h2-1],[w2-1,h2-1],[w2-1,0]]).reshape(-1,1,2)
 
#获取img1的边界坐标变换之后的坐标
img1_transform = cv2.perspectiveTransform(img1_pts,H)
# print('img1_pts',img1_pts)
# print('img1_transform',img1_transform)
 
#把img2和转换后的边界坐标连接起来
result_pts = np.concatenate((img2_pts,img1_transform),axis=0)
print(result_pts)
#result_pts.min(axis=0)  #对行聚合，返回每一列的最小值
[x_min,y_min] = np.int32(result_pts.min(axis=0).ravel()-1)
[x_max,y_max] = np.int32(result_pts.max(axis=0).ravel()+1)
 
#ret = cv2.drawMatchesKnn(img1,kp1,img2,kp2,[goods],None)
 
 
#手动构造平移矩阵
M = np.array([[1,0,-x_min],[0,1,-y_min],[0,0,1]])
result = cv2.warpPerspective(img1,M.dot(H),(x_max-x_min,y_max-y_min))  #对img1进行平移和透视操作
 
result[-y_min:-y_min+h2,-x_min:-x_min+w2] = img2   #把img2放进来(因为img1变换后的矩阵也平移了，所以img2也要做对应的平移)
 
#result[0:h2,0:w2] = img2
# cv2.imshow('direct',direct)
# cv2.imshow('warpimg',warpimg)
# cv2.imshow('ret',ret)
cv2.imshow('result',result)
cv2.waitKey(0)
cv2.destroyAllWindows()

goods 376
[[[   0.         0.     ]]

 [[   0.       649.     ]]

 [[ 499.       649.     ]]

 [[ 499.         0.     ]]

 [[-378.56302 -231.372  ]]

 [[-604.49756  738.69977]]

 [[ 150.16425  648.6845 ]]

 [[ 253.82436   56.88591]]]
